<a href="https://colab.research.google.com/github/nvhmadridista/BrainTumorDiagnosisUsingMRI/blob/feature%2Fmodel-training/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
PROJECT_DIR = '/content/drive/MyDrive/ML'
MODELS_DIR = os.path.join(PROJECT_DIR, 'models')
LOGS_DIR = os.path.join(PROJECT_DIR, 'logs')
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)
print("Đã gắn Drive và thiết lập thư mục lưu trữ.")

Mounted at /content/drive
Đã gắn Drive và thiết lập thư mục lưu trữ.


In [2]:
!git clone -b feature/model-training https://github.com/nvhmadridista/BrainTumorDiagnosisUsingMRI.git ML
%cd ML
!git branch

Cloning into 'ML'...
remote: Enumerating objects: 12391, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 12391 (delta 11), reused 37 (delta 8), pack-reused 12349 (from 1)
Receiving objects: 100% (12391/12391), 206.84 MiB | 16.32 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/ML
* feature/model-training


In [2]:
REPO_URL = f'https://github.com/nvhmadridista/BrainTumorDiagnosisUsingMRI/tree/feature/model-training'
REPO_NAME = 'ML'
!git clone {REPO_URL}
%cd {REPO_NAME}


Cloning into 'model-training'...
fatal: repository 'https://github.com/nvhmadridista/BrainTumorDiagnosisUsingMRI/tree/feature/model-training/' not found
[Errno 2] No such file or directory: 'ML'
/content


In [3]:
%cd /content/ML
DRIVE_ZIP_PATH = '/content/drive/MyDrive/KaggleData/archive.zip'
DATASET_DIR = 'dataset'
os.makedirs(DATASET_DIR, exist_ok=True)
!unzip -q {DRIVE_ZIP_PATH} -d {DATASET_DIR}
print("Đã giải nén dữ liệu gốc vào thư mục: /content/ML/dataset")

/content/ML
Đã giải nén dữ liệu gốc vào thư mục: /content/ML/dataset


In [4]:
from pathlib import Path
import math, shutil
import os
# Resplit images in dataset
BASE = Path("dataset")
SRC_TRAIN = BASE / "Training"
SRC_TEST = BASE / "Testing"
OUT_BASE = Path("data_split")
OUT_TRAIN = OUT_BASE / "train"
OUT_VAL = OUT_BASE / "val"
OUT_TEST = OUT_BASE / "test"

OUT_BASE.mkdir(parents=True, exist_ok=True)

classes = sorted({p.name for p in SRC_TRAIN.iterdir() if p.is_dir()} | {p.name for p in SRC_TEST.iterdir() if p.is_dir()})
for c in classes:
    files = []
    ct = SRC_TRAIN / c
    if ct.exists():
        files += sorted(ct.rglob("*.jpg"))
    cte = SRC_TEST / c
    if cte.exists():
        files += sorted(cte.rglob("*.jpg"))
    n = len(files)
    n_train = math.floor(n * 0.70)
    n_val = math.floor(n * 0.15)
    n_test = n - n_train - n_val
    splits = [(OUT_TRAIN / c, files[:n_train]), (OUT_VAL / c, files[n_train:n_train+n_val]), (OUT_TEST / c, files[n_train+n_val:])]
    for out_dir, group in splits:
        out_dir.mkdir(parents=True, exist_ok=True)
        for f in group:
            dst = out_dir / f.name
            if f.resolve() == dst.resolve():
                continue
            shutil.move(str(f), str(dst))

In [5]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), 'src/preprocessing')))
from run_preprocessing import run_main_preprocess
if __name__ == "__main__":
    tasks = [
        ("data_split/train", "data/processed/Training"),
        ("data_split/val", "data/processed/Validation"),
        ("data_split/test", "data/processed/Testing"),
    ]
os.makedirs("data/processed", exist_ok=True)

for raw_dir, out_dir in tasks:
    print(f"\n RUNNING PREPROCESSING: {raw_dir} - {out_dir} ")
    run_main_preprocess(raw_dir, out_dir)

print("\n All Preprocessing complete. Data ready for Training.")


 RUNNING PREPROCESSING: data_split/train - data/processed/Training 
Preprocessing class: pituitary


100%|██████████| 1229/1229 [00:12<00:00, 99.43it/s] 


Preprocessing class: meningioma


100%|██████████| 1151/1151 [00:11<00:00, 99.41it/s] 


Preprocessing class: glioma


...

Preprocessing class: glioma


100%|██████████| 244/244 [00:02<00:00, 112.66it/s]


Preprocessing class: notumor


100%|██████████| 300/300 [00:03<00:00, 93.94it/s]


 DONE! Preprocessed images from data_split/test saved in data/processed/Testing

 All Preprocessing complete. Data ready for Training.


In [6]:
!python -m src.training.train

Device: cuda
Đã lưu tên lớp: /content/drive/MyDrive/ML/models/class_names.json

Epoch 1/25
Train Loss: 6.7123 | Train Acc: 0.3451
Val Loss:   1.5825 | Val Acc:   0.3930
Saved best model! Val Loss: 1.5825

Epoch 2/25
Train Loss: 1.9898 | Train Acc: 0.4397
Val Loss:   1.2535 | Val Acc:   0.5203
Saved best model! Val Loss: 1.2535

Epoch 3/25
Train Loss: 1.3962 | Train Acc: 0.5728
Val Loss:   1.1433 | Val Acc:   0.6721
Saved best model! Val Loss: 1.1433

Epoch 4/25
Train Loss: 1.0053 | Train Acc: 0.7009
Val Loss:   0.6553 | Val Acc:   0.8062
Saved best model! Val Loss: 0.6553

Epoch 5/25
Train Loss: 0.7549 | Train Acc: 0.7720
Val Loss:   0.8116 | Val Acc:   0.7425

Epoch 6/25
Train Loss: 0.5886 | Train Acc: 0.8182
Val Loss:   0.5548 | Val Acc:   0.8144
Saved best model! Val Loss: 0.5548

Epoch 7/25
Train Loss: 0.4304 | Train Acc: 0.8669
Val Loss:   0.4757 | Val Acc:   0.8604
Saved best model! Val Loss: 0.4757

Epoch 8/25
Train Loss: 0.3755 | Train Acc: 0.8846
Val Loss:   0.3623 | Val Acc: 

In [7]:
%cd /content/ML
ZIP_FILENAME = "brain_data_raw_and_split.zip"
!zip -r -q {ZIP_FILENAME}  data_split/ data/processed/
print(f"Đã tạo file nén chứa tất cả dữ liệu: {ZIP_FILENAME}")

/content/ML
Đã tạo file nén chứa tất cả dữ liệu: brain_data_raw_and_split.zip


In [8]:
DRIVE_DATA_BACKUP = '/content/drive/MyDrive/ML/Data_Backup'
!mkdir -p {DRIVE_DATA_BACKUP}
!cp {ZIP_FILENAME} {DRIVE_DATA_BACKUP}/
print(f"Đã sao chép file nén lên Drive thành công tại: {DRIVE_DATA_BACKUP}/{ZIP_FILENAME}")

Đã sao chép file nén lên Drive thành công tại: /content/drive/MyDrive/ML/Data_Backup/brain_data_raw_and_split.zip
